In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.datasets import mnist
import time
import urllib
import os.path
import sys
import os
from os import listdir
from tensorflow.keras.utils import img_to_array, load_img
from numpy import asarray
from keras.preprocessing.image import ImageDataGenerator

In [15]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it2 = datagen.flow_from_directory('/Users/jcsansom/Documents/562 final/image_to_prompt/archive_g/Training/',
 class_mode='categorical', batch_size=10, target_size=(150, 150), color_mode="grayscale")
test_it2 = datagen.flow_from_directory('/Users/jcsansom/Documents/562 final/image_to_prompt/archive_g/Testing/',
 class_mode='categorical', batch_size=10, target_size=(150, 150), color_mode="grayscale")

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [18]:
model2 = keras.Sequential()
model2.add(Conv2D(32, (5, 5), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(150, 150, 1)))
model2.add(MaxPooling2D(2, 2))
model2.add(BatchNormalization())
model2.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model2.add(Dropout(0.3))
model2.add(MaxPooling2D(2, 2))
model2.add(BatchNormalization())
model2.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model2.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(4, activation='softmax'))

model2.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
model2.fit_generator(train_it2, steps_per_epoch=len(train_it2), validation_data=test_it2, validation_steps=len(test_it), epochs=10, verbose=1)
model2.summary()
#model.save('/Users/jcsansom/Documents/562 final/image_to_prompt/')
score = model2.evaluate_generator(test_it2, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/10


/var/folders/rc/gp38btvd4t10gf8g1hgvk0s40000gn/T/ipykernel_37103/3648083685.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2.fit_generator(train_it2, steps_per_epoch=len(train_it2), validation_data=test_it2, validation_steps=len(test_it), epochs=10, verbose=1)


287/287 [==============================] - 27s 94ms/step - loss: 0.8604 - accuracy: 0.6976 - val_loss: 1.6419 - val_accuracy: 0.4567
Epoch 2/10
287/287 [==============================] - 28s 98ms/step - loss: 0.3291 - accuracy: 0.8672 - val_loss: 1.5192 - val_accuracy: 0.6800
Epoch 3/10
287/287 [==============================] - 29s 101ms/step - loss: 0.1621 - accuracy: 0.9432 - val_loss: 2.1914 - val_accuracy: 0.7000
Epoch 4/10
287/287 [==============================] - 32s 111ms/step - loss: 0.1019 - accuracy: 0.9627 - val_loss: 1.8957 - val_accuracy: 0.7400
Epoch 5/10
287/287 [==============================] - 29s 101ms/step - loss: 0.0268 - accuracy: 0.9902 - val_loss: 3.1456 - val_accuracy: 0.7100
Epoch 6/10
287/287 [==============================] - 29s 100ms/step - loss: 0.1314 - accuracy: 0.9578 - val_loss: 3.6359 - val_accuracy: 0.7200
Epoch 7/10
287/287 [==============================] - 30s 103ms/step - loss: 0.0498 - accuracy: 0.9826 - val_loss: 2.8061 - val_accuracy: 0.746

/var/folders/rc/gp38btvd4t10gf8g1hgvk0s40000gn/T/ipykernel_37103/3648083685.py:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model2.evaluate_generator(test_it2, verbose=1)


40/40 [==============================] - 2s 44ms/step - loss: 3.1319 - accuracy: 0.7386
Test loss: 3.1318962574005127
Test accuracy: 0.7385786771774292


In [20]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it = datagen.flow_from_directory('/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Training/',
 class_mode='categorical', batch_size=10, target_size=(150, 150), color_mode="grayscale")
test_it = datagen.flow_from_directory('/Users/jcsansom/Documents/562 final/image_to_prompt/archive/Testing/',
 class_mode='categorical', batch_size=10, target_size=(150, 150), color_mode="grayscale")

Found 2044 images belonging to 3 classes.
Found 294 images belonging to 3 classes.


In [21]:
model = keras.Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(150, 150, 1)))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
model.fit_generator(train_it, steps_per_epoch=len(train_it), validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1)
model.summary()
#model.save('/Users/jcsansom/Documents/562 final/image_to_prompt/')
score = model.evaluate_generator(test_it, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/10


/var/folders/rc/gp38btvd4t10gf8g1hgvk0s40000gn/T/ipykernel_37103/2347890943.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_it, steps_per_epoch=len(train_it), validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1)


205/205 [==============================] - 22s 109ms/step - loss: 0.0272 - accuracy: 0.9946 - val_loss: 0.5052 - val_accuracy: 0.9014
Epoch 2/10
205/205 [==============================] - 24s 116ms/step - loss: 0.0485 - accuracy: 0.9853 - val_loss: 0.6600 - val_accuracy: 0.9014
Epoch 3/10
205/205 [==============================] - 24s 115ms/step - loss: 0.0667 - accuracy: 0.9814 - val_loss: 0.2987 - val_accuracy: 0.9218
Epoch 4/10
205/205 [==============================] - 24s 117ms/step - loss: 0.0523 - accuracy: 0.9878 - val_loss: 0.3450 - val_accuracy: 0.9218
Epoch 5/10
205/205 [==============================] - 23s 111ms/step - loss: 0.0248 - accuracy: 0.9927 - val_loss: 0.3120 - val_accuracy: 0.9150
Epoch 6/10
205/205 [==============================] - 23s 113ms/step - loss: 0.0409 - accuracy: 0.9917 - val_loss: 0.5088 - val_accuracy: 0.8776
Epoch 7/10
205/205 [==============================] - 24s 117ms/step - loss: 0.0243 - accuracy: 0.9927 - val_loss: 0.5793 - val_accuracy: 0.9

/var/folders/rc/gp38btvd4t10gf8g1hgvk0s40000gn/T/ipykernel_37103/2347890943.py:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(test_it, verbose=1)


30/30 [==============================] - 1s 34ms/step - loss: 0.0917 - accuracy: 0.9830
Test loss: 0.09174088388681412
Test accuracy: 0.9829931855201721
